In [1]:
using NLOptControl
using Polynomials
using Plots
using Parameters
pyplot()
############
# TEST DATA
############
t0 = Float64(0); tf = Float64(2);
t = Array(linspace(t0,tf,100));

# sin and cos taylor series approximation at x = 0
s0 =  0; s1 = 1; s2 = 0;       s3 = -1/(3*2*1); s4 = 0;           s5 = 1/(5*4*3*2*1); 
c0 = 1;  c1 = 0; c2 = 1/(2*1); c3 = 0;          c4 = 1/(4*3*2*1); c5 = 0; 
γ1 = Poly([s0,s1,s2,s3,s4,s5]);  # state 1
γ2 = Poly([c0,c1,c2,c3,c4,c5]);  # state 2
γ = [γ1; γ2]; numStates=length(γ);
y = [polyval(γ[st],t) for st in 1:numStates];

# evaluate the integrals
∫γ = [polyint(γ[st]) for st in 1:numStates];
Y  = [polyval(∫γ[st],t[end]) - polyval(∫γ[st],t[1]) for st in 1:numStates];
C  = [Y[st] - polyval(∫γ[st],t[end]) for st in 1:numStates]; # constant of integration
∫y = [polyval(∫γ[st],t) + C[st] for st in 1:numStates];

# evaluate the derivatives
dγ = [polyder(γ[st]) for st in 1:numStates];
dy = [polyval(dγ[st],t) for st in 1:numStates];
############
# TEST DATA
############
dx1(t) = 1.0 - 0.5⋅t^2 + 0.0416667⋅t^4;   
dx2(t) = 1.0⋅t + 0.166667⋅t^3;
dx = [dx1, dx2];
X0= [y[1][1];y[2][1]];
XF= [y[1][end];y[2][end]];
XL=[0,-Inf];     XU=[1/9,Inf]; # TODO allow for functions of these so we can calculate them on the fly!
CL=[-Inf];  CU=[Inf]; 

ps, nlp = initialize_NLP(numStates=2,numControls=1,Ni=5,Nck=[4,4,4,4,4],stateEquations=dx,X0=X0,XF=XF,XL=XL,XU=XU,CL=CL,CU=CU);

@pack ps = t0, tf;  # given in problem def.
@unpack Nck, Ni, t0, tf, τ, ω = ps;
di, tm, ts, ωₛ = create_intervals(t0,tf,Ni,Nck,τ,ω);
@pack ps = τ, ω, ωₛ, ts;
generate_Fake_data(nlp,ps,γ);
nlp2ocp(nlp,ps);

@unpack stateMatrix = ps;
@unpack numStates = nlp;

# interpolate function using Lagrange Polynomial
P = [zeros(Float64,Nck[int]+1,numStates) for int in 1:Ni];
for st in 1:numStates
    for int in 1:Ni
        P[int][:,st] = interpolate_lagrange(ts[int],ts[int],stateMatrix[int][1:end,st],Nck[int])
    end
end

# approximate integral using quadrature
ζ, approx_int_st = integrate_state(ps,nlp)

# calculate LGR matrices - > IMatrix and DMatrix
LGR_matrices(ps,nlp)

# approximate integral using LGRIM
ζ2, approx_int_st2 = integrate_state(ps,nlp;(:mode=>:LGRIM))

# approximate derivative using LGRDM
dζ = differentiate_state(ps,nlp);
#################

5-element Array{Array{Float64,2},1}:
 [0.999976 0.996405 0.972225 0.934286; 5.32907e-15 0.0850383 0.23841 0.37264]
 [0.921042 0.884732 0.804437 0.721962; 0.410667 0.503943 0.679133 0.839054]  
 [0.697042 0.634006 0.511163 0.398536; 0.885333 1.00044 1.22165 1.4278]      
 [0.366376 0.288062 0.145921 0.0257404; 1.488 1.63852 1.92996 2.20287]       
 [-0.00695771 -0.0836632 -0.21217 -0.309093; 2.28267 2.48219 2.86807 3.22828]

In [5]:
c, ceq = constraints(nlp,ps);

In [6]:
c

140-element Array{Float64,1}:
    0.0      
   -0.0848341
   -0.234023 
   -0.356543 
   -0.389419 
   -0.389419 
   -0.466153 
   -0.594162 
   -0.692253 
   -0.717397 
   -0.717397 
   -0.773958 
   -0.860732 
    ⋮        
 -Inf        
 -Inf        
 -Inf        
 -Inf        
 -Inf        
 -Inf        
 -Inf        
 -Inf        
 -Inf        
 -Inf        
 -Inf        
 -Inf        

In [7]:
ceq

56-element Array{Float64,1}:
 -2.4381e-5  
  9.43332e-6 
 -6.26013e-6 
  3.86318e-6 
  5.47307e-15
 -2.0425e-10 
 -4.3933e-9  
 -1.61511e-8 
 -2.43818e-5 
  9.43148e-6 
 -6.26548e-6 
  3.85238e-6 
 -2.13333e-8 
  ⋮          
  0.0        
  0.0        
  0.0        
  0.0        
  0.0        
  0.0        
  0.0        
  0.0        
  0.0        
  0.0        
  0.0        
  0.0        

In [3]:
# can do these in parallel -> they do not depend on each other
ode_constraint(nlp,ps);
continuity_constraint(nlp,ps);
boundary_constraint(nlp,ps);
inequality_constraint(nlp,ps);

@unpack odeConstraint, continuityConstraint, boundaryConstraint, stateConstraint, controlConstraint = ps

ceq1 = [idx for tempM in odeConstraint for idx = tempM];
ceq2 = [idx for tempM in continuityConstraint for idx = tempM];
ceq3 = [idx for tempM in boundaryConstraint for idx = tempM];
ceq = [ceq1; ceq2; ceq3];

c1 = [idx for tempM in stateConstraint for idx = tempM];
c2 = [idx for tempM in controlConstraint for idx = tempM];
c = [c1; c2];

In [1]:
function constraint_vector(nlp::NLP_data,ps::PS_data)
    @unpack c, cineq = nlp
end

LoadError: UndefVarError: @unpack not defined